# Test the transformation on data from the actual problem.

We developed the new equality constraint

$\sum V_{\Delta} \cdot P = c ||P||_1$

Our goal is to take elements of a space

$V_{\Delta}''$ where $l \leq (V_{\Delta})_i \leq u$ for all $i$

and transform into a vector satisfying the condition above.

(Remember we go from $n-1$ dimensional space to $n$ dimensional space.)

In [1]:
# load libraries
import numpy as np
import pandas as pd
import scipy

In [3]:
# load and clean data
with open("../project_dir.txt") as f:
    project_path = f.readline().strip() + "{}"
    
print(project_path)
# vaccination data and population data from sifat
vacc_df = pd.read_csv(project_path.format("data/VA_zipcodes_cleaned/ZC_immunization_sifat.csv"))
# drop 0 population entries, they won't affect the simulation
vacc_df = vacc_df[vacc_df['population'] > 0].reset_index(drop=True)
vacc_df.rename({'population':'pop'},axis=1,inplace=True)
# load distance matrix computed from nominatim and geopy distance function
dist_df = pd.read_csv(project_path.format("data/VA_zipcodes_cleaned/ZC_distance_sifat_nom_geopy.csv"))
# need to replace 0's in distance matrix to avoid divide by zero in gravity formula
default_dist = 0.5
dist_df.loc[dist_df[np.isclose(dist_df['distKM'],0)].index,'distKM']=default_dist
# convert to matrix
dist_mat = dist_df.pivot(index='zipcode1',columns='zipcode2',values='distKM')
dist_mat = dist_mat.replace(np.nan,0)
# align matrix
dist_mat = dist_mat.loc[vacc_df['zipcode'],vacc_df['zipcode']]

top_5 = vacc_df.sort_values(by='pop',ascending=False).head(5)
I = np.zeros(len(vacc_df.index))
np.put(I,top_5.index,1)


/home/nick/Documents/4tb_sync/UVA GDrive/Summer 2022 (C4GC with BII)/measles_metapop/{}


In [7]:
constraint_bnd = 0.05
V_0 = np.array((vacc_df['pop'] - vacc_df['nVaccCount'])/(vacc_df['pop']))
dim = len(V_0)
print(dim)
eq_constraint_bnd = constraint_bnd*sum(vacc_df['pop'])
P =np.array(vacc_df['pop'])

705


# check the forward transformation again

In [97]:
def generate_v_delta_candidate(V_0):
    V_delta = np.zeros(len(V_0))
    for i,value in enumerate(V_0):
        V_delta[i] = np.random.random()*value
    return V_delta

In [98]:
# doing the change of variable
test_point = generate_v_delta_candidate(V_0)*np.array(vacc_df['pop'])

In [100]:
sum(test_point)

3433192.5305393278

In [42]:
def generate_number_summing_to(sum_target,dim,n_points):
    # better way of generating random numbers?
    points = []
    while len(points) < n_points:
        x = [np.random.random() for i in range(dim)]
        x.insert(0,0)
        x.append(1)
        x.sort()
        points.append(np.diff(x)*sum_target)
    points = np.float128(points)
    return points

In [43]:
def generate_number_sum_(sum_target,dim,n_points):
    # better way of generating random numbers?
    points = []
    while len(points) < n_points:
        x = [np.random.random() for i in range(dim)]
        x.insert(0,0)
        x.append(1)
        x.sort()
        points.append(np.diff(x)*sum_target)
    points = np.float128(points)
    return points

In [44]:
generate_number_summing_to(eq_constraint_bnd,dim-1,1)[0]/vacc_df['pop']

0      0.086172
1      0.715136
2      0.047500
3      0.002825
4      0.196103
         ...   
700    0.460563
701    0.017579
702    0.024471
703    0.062349
704    0.005594
Name: pop, Length: 705, dtype: float128

In [73]:
# now let's generate some candidate vectors in our new space

# todo: what to choose for lower and upper bounds? idk
l = 10
u = 110
n_points = 100
points = np.array([np.array([np.random.random()*(u-l)+l for i in range(dim-1)]) for i in range(n_points)])
len(points[0])

704

In [33]:
# maybe we can make it better by posing it as a linear system
# and hoping that the np solver does something smart
from scipy.optimize import root
def inv_tr_lin_solve_eq(point,l,u,c):
    dim = len(point)
    coeff_matrix = np.eye(dim+1)
    # add bottom row of ones
    #coeff_matrix[dim-1,:] = np.ones(dim)
    # add coefficients to off-diagonal
    for i in range(0,dim):
        coeff_matrix[i,i+1] = (l-point[i])/(u-l)
    #print(coeff_matrix)
    b = np.zeros(dim+1)
    b[dim] = c
    #print(b)
    L = np.tril(np.ones((dim+1,dim+1)))
    x = scipy.linalg.solve(coeff_matrix@L,b)
    return x
    #def objective(x):
    #    return (coeff_matrix@L)@x-b
    #return root(objective,np.ones(dim))

In [75]:
inv_tr_lin_solve_eq(points[0],l,u,c=eq_constraint_bnd)

array([-3.66485408e-11,  1.89968837e-11, -2.31837744e-11, -2.41233671e-11,
        2.82170981e-12,  4.42379930e-11, -1.18088399e-11,  2.76445502e-11,
       -1.36512641e-11, -5.12624569e-13,  1.38152309e-11, -6.81853636e-13,
       -3.18808740e-12, -2.63615635e-11,  2.77707275e-11, -4.28262179e-12,
       -1.64853419e-12, -2.31080100e-12, -2.30028792e-11, -2.30014900e-11,
        5.44288895e-11,  1.86128052e-11,  1.76555037e-11, -1.07435328e-11,
       -3.12533015e-11,  6.04999245e-11, -1.08679494e-10,  1.58843858e-11,
       -8.88409844e-13,  1.71464260e-11, -1.64119030e-12,  3.01569048e-11,
        3.76750549e-12,  7.43063678e-12,  2.07070217e-11,  2.31792849e-11,
       -6.49146507e-11, -4.34481578e-12, -1.03348670e-11, -5.79010471e-12,
        1.08963721e-11, -6.78104386e-12, -2.60896972e-11, -8.19723397e-12,
        7.22157572e-13,  7.25070751e-11, -4.09901064e-11,  1.19340024e-11,
       -2.78180338e-12,  1.40065680e-11, -8.15889698e-12, -7.48819203e-12,
       -4.21879171e-11,  

# sandboxing another constraint

In [38]:
S_0

array([1.364000e+03, 2.878000e+03, 3.866000e+03, 1.603200e+04,
       1.821600e+04, 2.433000e+04, 2.667600e+04, 2.820800e+04,
       2.966800e+04, 6.368100e+04, 6.693500e+04, 1.013480e+05,
       1.293510e+05, 1.432980e+05, 1.900810e+05, 2.108690e+05,
       2.280960e+05, 2.291840e+05, 2.489260e+05, 2.574370e+05,
       2.641950e+05, 2.651620e+05, 2.686720e+05, 2.706820e+05,
       2.718370e+05, 2.947990e+05, 3.315110e+05, 3.782620e+05,
       4.098150e+05, 4.552500e+05, 4.612530e+05, 4.674730e+05,
       4.905680e+05, 4.966860e+05, 5.024270e+05, 5.044210e+05,
       5.054110e+05, 5.158170e+05, 5.179220e+05, 5.186300e+05,
       5.250780e+05, 5.256500e+05, 5.395230e+05, 5.547050e+05,
       5.697420e+05, 5.952250e+05, 6.049290e+05, 6.059650e+05,
       6.544660e+05, 6.556980e+05, 6.573430e+05, 6.595460e+05,
       6.601990e+05, 6.621910e+05, 6.961950e+05, 7.368750e+05,
       7.633100e+05, 7.652040e+05, 7.677630e+05, 7.699890e+05,
       7.704140e+05, 7.710570e+05, 8.099770e+05, 8.6322

In [37]:
v_0_prime = V_0*P
S_0 = np.cumsum(v_0_prime)
new_upbound = np.array([
    np.sum((V_0*P)[0:k])/(eq_constraint_bnd - np.sum((V_0*P)[0:len(P)- k-1]))
    for k in range(len(P)-1)
])
new_upbound = new_upbound*100
print(new_upbound[0:15])

[-0.         -0.02133868 -0.04526734 -0.06086296 -0.25249311 -0.28841595
 -0.3852471  -0.4224529  -0.44690577 -0.47019593 -1.00961976 -1.06178783
 -1.60830116 -2.05391942 -2.2769007 ]


In [41]:
eq_constraint_bnd-S_0

array([ 3.7083090e+05,  3.6931690e+05,  3.6832890e+05,  3.5616290e+05,
        3.5397890e+05,  3.4786490e+05,  3.4551890e+05,  3.4398690e+05,
        3.4252690e+05,  3.0851390e+05,  3.0525990e+05,  2.7084690e+05,
        2.4284390e+05,  2.2889690e+05,  1.8211390e+05,  1.6132590e+05,
        1.4409890e+05,  1.4301090e+05,  1.2326890e+05,  1.1475790e+05,
        1.0799990e+05,  1.0703290e+05,  1.0352290e+05,  1.0151290e+05,
        1.0035790e+05,  7.7395900e+04,  4.0683900e+04, -6.0671000e+03,
       -3.7620100e+04, -8.3055100e+04, -8.9058100e+04, -9.5278100e+04,
       -1.1837310e+05, -1.2449110e+05, -1.3023210e+05, -1.3222610e+05,
       -1.3321610e+05, -1.4362210e+05, -1.4572710e+05, -1.4643510e+05,
       -1.5288310e+05, -1.5345510e+05, -1.6732810e+05, -1.8251010e+05,
       -1.9754710e+05, -2.2303010e+05, -2.3273410e+05, -2.3377010e+05,
       -2.8227110e+05, -2.8350310e+05, -2.8514810e+05, -2.8735110e+05,
       -2.8800410e+05, -2.8999610e+05, -3.2400010e+05, -3.6468010e+05,
      

In [30]:
len(new_upbound)

704

In [31]:
test_v = np.array([np.random.random()*k for k in new_upbound])

In [35]:
inv_tr_lin_solve_eq(test_v,0,100,c=eq_constraint_bnd)*P

array([-8.61350568e-08,  9.62479340e-08, -1.91687570e-10, -7.60477567e-07,
        1.36550103e-07, -3.85961500e-07,  5.55127244e-10,  9.83536466e-08,
       -1.53731350e-10, -2.01265236e-06,  1.89863897e-07, -4.28525411e-06,
        3.46575309e-06, -8.69816335e-07,  3.21763373e-06, -5.77286805e-08,
       -5.94687627e-08,  7.84325328e-11, -4.92271810e-08, -1.05867781e-06,
        4.27070150e-07,  9.44420186e-11,  2.23605880e-07, -4.54287411e-10,
       -7.36908028e-08,  2.89999086e-09,  9.98051806e-08,  3.06703748e-06,
       -1.55041238e-07, -2.79187962e-06,  3.72411731e-07, -6.82959476e-09,
       -2.92979238e-06,  3.87721239e-07,  3.51207144e-12,  1.93232092e-10,
        6.41639244e-08, -6.62636012e-07,  6.91013538e-16,  5.02886755e-13,
        3.05307713e-09,  3.63974403e-08, -8.79335606e-07,  2.83489368e-10,
        1.71534140e-08,  1.63098428e-06, -6.38772610e-07, -6.48217323e-08,
        3.25972451e-06,  7.40745962e-08, -1.06578227e-07,  1.30009135e-16,
        7.01531582e-15,  